# data_proc.ipynb 
### Made by William Rubio on June 2023

In [2]:
# Download the dataset
!wget -P data/ https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt 

--2023-06-14 13:31:36--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'data/input.txt.2'

input.txt.2         100%[===================>]   1.06M   612KB/s    in 1.8s    

2023-06-14 13:31:39 (612 KB/s) - 'data/input.txt.2' saved [1115394/1115394]



In [3]:
# Read the dataset and save it in a variable
with open('data/input.txt', 'r') as f:
    text = f.read()
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
# Inspect the characters that are present in the dataset
chars = sorted(list(set(text)))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


## Tokens

Tokenize input text: Covert raw text as a string to a sequence of integers according to a vocabulary of possible elements. 
Google uses SentencePiece, it's a sub-word tokenizer, ++ Vocabulaty == ++ Tokens == -- Decode array == More reasoning

In [5]:
stoi = []
i = 0
for character in chars:
    stoi.append([i, character])
    i += 1
print(stoi)

[[0, '\n'], [1, ' '], [2, '!'], [3, '$'], [4, '&'], [5, "'"], [6, ','], [7, '-'], [8, '.'], [9, '3'], [10, ':'], [11, ';'], [12, '?'], [13, 'A'], [14, 'B'], [15, 'C'], [16, 'D'], [17, 'E'], [18, 'F'], [19, 'G'], [20, 'H'], [21, 'I'], [22, 'J'], [23, 'K'], [24, 'L'], [25, 'M'], [26, 'N'], [27, 'O'], [28, 'P'], [29, 'Q'], [30, 'R'], [31, 'S'], [32, 'T'], [33, 'U'], [34, 'V'], [35, 'W'], [36, 'X'], [37, 'Y'], [38, 'Z'], [39, 'a'], [40, 'b'], [41, 'c'], [42, 'd'], [43, 'e'], [44, 'f'], [45, 'g'], [46, 'h'], [47, 'i'], [48, 'j'], [49, 'k'], [50, 'l'], [51, 'm'], [52, 'n'], [53, 'o'], [54, 'p'], [55, 'q'], [56, 'r'], [57, 's'], [58, 't'], [59, 'u'], [60, 'v'], [61, 'w'], [62, 'x'], [63, 'y'], [64, 'z']]


In [6]:
# Encode and decode functions
def encode(string):
    ans = []
    for character in string:
        for tuple in stoi:
            if tuple[1] == character:
                ans.append(tuple[0])
                break
    return ans

def decode(arr):
    ans = ""
    for num in arr:
        for tuple in stoi:
            if tuple[0] == num:
                ans+=tuple[1]
                break
    return ans



In [7]:
# Encode dataset
import torch as pt
data = pt.tensor(encode(text), dtype=pt.long)
print(data.shape)
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
# Data split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

## Batch prep
Computationally, it is not efficient to train with all the data at once. Because of this, we dide it in chunks (blocks). The reason why this is way more efficioent is because machines are good at concurrent processing, and we should be using all cores in a CPU.

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(context, "->", target)

tensor([18]) -> tensor(47)
tensor([18, 47]) -> tensor(56)
tensor([18, 47, 56]) -> tensor(57)
tensor([18, 47, 56, 57]) -> tensor(58)
tensor([18, 47, 56, 57, 58]) -> tensor(1)
tensor([18, 47, 56, 57, 58,  1]) -> tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) -> tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> tensor(58)


Quick explanation of the code below: each row of the input x is a chunk of the training set. Each row of the target y is the number it should predict given the input in the corresponding placement of x. For example, looking at the first row and column element of x and y, we can see that 24 is likely to produce 43, 24 + 43 is likely to produce 58. Etc, etc. 

In [13]:
import torch
torch.manual_seed(1337)
batch_size = 4 # Sequences to process in parallel
block_size = 8 # Maximum context length for predictions

def get_batch(split):
    """ Generate a small batch of data of inputs x and targets y"""
    data = train_data if split == 'train' else val_data
    # Generate batch_size random integers from 0 to len(data)-block_size, this will be the random offset for each sequence in the batch
    ix = torch.randint(len(data) - block_size, (batch_size,)) 
    x = torch.stack([data[i:i+block_size] for i in ix]) # Stack the sequences (they have different lengths)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # Shifted version of the sequences for the targets
    return x, y

xb, yb = get_batch('train')
print("inputs: ")
print(xb.shape)
print(xb)
print("targets: ")
print(yb.shape)
print(yb)

inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


## Language model training
We will start with the pytorch bigram language model. Quick definition: A bigram language model is a word prediction mathematical model that predicts the next word in a sentence based on the previous words. It achieves by analyzing patterns observed in large collections of texts.

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # The token embedding table is lookup table that maps each word or token to its corresponding vector representation
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)